In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection  import train_test_split
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



/kaggle/input/breast-cancer-coimbra-data-set/dataR2.csv
/kaggle/input/water-potability/water_potability.csv


# Veri Ön İşlemesi

In [2]:
df = pd.read_csv('../input/water-potability/water_potability.csv')
print(df)

            ph    Hardness        Solids  Chloramines     Sulfate  \
0          NaN  204.890455  20791.318981     7.300212  368.516441   
1     3.716080  129.422921  18630.057858     6.635246         NaN   
2     8.099124  224.236259  19909.541732     9.275884         NaN   
3     8.316766  214.373394  22018.417441     8.059332  356.886136   
4     9.092223  181.101509  17978.986339     6.546600  310.135738   
...        ...         ...           ...          ...         ...   
3271  4.668102  193.681735  47580.991603     7.166639  359.948574   
3272  7.808856  193.553212  17329.802160     8.061362         NaN   
3273  9.419510  175.762646  33155.578218     7.350233         NaN   
3274  5.126763  230.603758  11983.869376     6.303357         NaN   
3275  7.874671  195.102299  17404.177061     7.509306         NaN   

      Conductivity  Organic_carbon  Trihalomethanes  Turbidity  Potability  
0       564.308654       10.379783        86.990970   2.963135           0  
1       592.88535

## b)Veri setinde tanımlı olmayan değerler bulmaya çalışıyoruz

In [3]:
df.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

## boş değerleri kaldırıyoruz.

In [4]:
fulldf=df.dropna()
print(fulldf)
fulldf.isnull().sum()

             ph    Hardness        Solids  Chloramines     Sulfate  \
3      8.316766  214.373394  22018.417441     8.059332  356.886136   
4      9.092223  181.101509  17978.986339     6.546600  310.135738   
5      5.584087  188.313324  28748.687739     7.544869  326.678363   
6     10.223862  248.071735  28749.716544     7.513408  393.663396   
7      8.635849  203.361523  13672.091764     4.563009  303.309771   
...         ...         ...           ...          ...         ...   
3267   8.989900  215.047358  15921.412018     6.297312  312.931022   
3268   6.702547  207.321086  17246.920347     7.708117  304.510230   
3269  11.491011   94.812545  37188.826022     9.263166  258.930600   
3270   6.069616  186.659040  26138.780191     7.747547  345.700257   
3271   4.668102  193.681735  47580.991603     7.166639  359.948574   

      Conductivity  Organic_carbon  Trihalomethanes  Turbidity  Potability  
3       363.266516       18.436524       100.341674   4.628771           0  
4    

ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64

## Veri normalize ediliyor

In [5]:
scaler = MinMaxScaler()
data = scaler.fit_transform(fulldf)
data = pd.DataFrame(data = data,columns = fulldf.columns)
data.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,0.587349,0.577747,0.386298,0.568199,0.647347,0.292985,0.654522,0.795029,0.630115,0.0
1,0.643654,0.441300,0.314381,0.439304,0.514545,0.356685,0.377248,0.202914,0.520358,0.0
2,0.388934,0.470876,0.506122,0.524364,0.561537,0.142913,0.249922,0.401487,0.219973,0.0
3,0.725820,0.715942,0.506141,0.521683,0.751819,0.148683,0.467200,0.658678,0.242428,0.0
4,0.610517,0.532588,0.237701,0.270288,0.495155,0.494792,0.409721,0.469762,0.585049,0.0


## Test ve train datası için kolonları ayrılır.

In [6]:
x = data.iloc[:,0:9]
y = data.iloc[:,-1]


## Temel bileşen analizine normalize edilmiş veri uygulanır.

In [7]:
pca = PCA(0.90) 
pca.fit(x)
x_pca = pca.transform(x)

## Temel bileşen analizi sonrasında kolon sayımız

In [8]:
x_pca.shape

(2011, 8)

## Test ve train datası ayrılıyor

In [9]:
x_train,x_test,y_train,y_test = train_test_split(x_pca,y,test_size=0.3,random_state=400)

## KNN uygulanıyor.

In [10]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train,y_train.ravel())

KNeighborsClassifier()

# Sonuç


In [11]:
result = knn.predict(x_test)
print(metrics.classification_report(y_test,result))

              precision    recall  f1-score   support

         0.0       0.64      0.72      0.68       368
         1.0       0.46      0.37      0.41       236

    accuracy                           0.58       604
   macro avg       0.55      0.55      0.54       604
weighted avg       0.57      0.58      0.57       604

